# STEP1.

In [40]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

    self.data = data  # 통상값
    self.grad = None  # 미분값(s6에서 역전파를 이용한 미분 구현을 위해 Variable class 확장)
    self.creator = None  # 인스턴스 변수 추가 (s7)
    
  def set_creator(self, func):  # creator를 설정할 수 있도록 메서드 추가 (s7)
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data) # 역전파 구현시마다 y.grad = np.array(1.) 부분을 쓰던 것을 생략해도 되게끔 개선

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()  # 함수를 가져온다
      x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
      x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

      if x.creator is not None:
        funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.

In [41]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [42]:
data = np.array(2.0)  # x는 데이터를 담은 상자(인스턴스)이기 때문에, 인스턴스 변수(x.data)가 아닌, 일반 변수 data에 새로운 값 할당해도 인스턴스(객체) 변수는 변경되지 않음.
print(x.data)

1.0


In [43]:
x.data = np.array(2.) # Variable 클래스를 상자로 이용
print(x.data)

2.0


In [44]:
import numpy as np
x = np.array(1)
x.ndim



0

In [45]:
x = np.array([1, 2, 3])
x.ndim

1

In [46]:
x = np.array([[1, 2, 3], 
             [4, 5, 6]])  # 2차원 배열, 대괄호 2개 임에 유의!
x.ndim

2

# STEP2.

In [47]:
class Function:
  def __call__(self, inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)  # 출력 변수에 창조자를 설정한다. -> 이 부분이 연결을 동적으로 만드는 기법의 핵심(DeZero의 동적 계산 그래프)
    self.inputs = inputs  # s6. 역전파 시 활용하기 위해 입력 변수 기억(보관)한다.
    self.outputs = outputs  # 출력도 저장한다.
    return outputs

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):  # s6. 역전파로 확장
    raise NotImplementedError()

In [48]:
class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y
  
  def backward(self, gy): # 역전파 메서드 추가확장
    x = self.input.data
    gx = 2 * x * gy
    return gx

In [49]:
'''
x = Variable(np.array(10))
f = Square() # 기존 f = Function() 에서 square() 함수로 세분화한 것에 따른 수정.
y = f(x)

print(type(y))
print(y.data)
'''

'\nx = Variable(np.array(10))\nf = Square() # 기존 f = Function() 에서 square() 함수로 세분화한 것에 따른 수정.\ny = f(x)\n\nprint(type(y))\nprint(y.data)\n'

# STEP3.

In [50]:
class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y

  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [51]:
'''
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
# print(y.data)

y.grad = np.array(1.)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)
'''

'\nA = Square()\nB = Exp()\nC = Square()\n\nx = Variable(np.array(0.5))\na = A(x)\nb = B(a)\ny = C(b)\n# print(y.data)\n\ny.grad = np.array(1.)\nb.grad = C.backward(y.grad)\na.grad = B.backward(b.grad)\nx.grad = A.backward(a.grad)\nprint(x.grad)\n'

# STEP4.

In [52]:
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)


In [53]:
'''
f = Square()
x = Variable(np.array(2.))
dy = numerical_diff(f, x)
print(dy)
'''

'\nf = Square()\nx = Variable(np.array(2.))\ndy = numerical_diff(f, x)\nprint(dy)\n'

In [54]:
'''
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)
'''

'\ndef f(x):\n  A = Square()\n  B = Exp()\n  C = Square()\n  return C(B(A(x)))\n\nx = Variable(np.array(0.5))\ndy = numerical_diff(f, x)\nprint(dy)\n'

# STEP6. 

# STEP7.

In [55]:
'''
# s7에서 수정된 Variable을 이용하여 역전파가 자동으로 실행됨을 확인.

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
# print(y.data)

y.grad = np.array(1.)
y.backward()
# b.grad = C.backward(y.grad)
# a.grad = B.backward(b.grad)
# x.grad = A.backward(a.grad)
print(x.grad)
'''

TypeError: ignored

# STEP8

# STEP9 함수를 더 편리하게

In [56]:
def square(x): # Square, Exp를 "class"로 구현해서 인스턴스 생성 및 인스턴스 호출 두 단계로 코딩해야 하는 불편함을 개선하기 위해 "함수"로 구현
  f = Square()
  return f(x)

def exp(x):
  f = Exp()
  return f(x)

In [57]:
'''
x = Variable(np.array(0.5))
y = square(exp(square(x)))  # 연속하여 적용도 가능
y.grad = np.array(1.)
y.backward()
print(x.grad)
'''

TypeError: ignored

In [58]:
'''
x = Variable(np.array(0.5))
y = square(exp(square(x)))  # 연속하여 적용도 가능

y.backward()
print(x.grad)
'''

TypeError: ignored

In [59]:
'''
x = Variable(np.array(1.))
x = Variable(None)

x = Variable(1.)

'''

TypeError: ignored

In [60]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [61]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [62]:
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return(x)

# STEP10 테스트

코랩에서 unittest 사용하는 방법을 모르겠음..
터미널 명령어를 어떻게 실행해야 하는지,
test 파일을 어느 경로에 둬야 하는지

# STEP11 가변 길이 인수(순전파 편)


In [63]:
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,)

In [64]:
xs = [Variable(np.array(2)), Variable(np.array(3))]  # 리스트로 준비
f = Add()
ys = f(xs)  # 튜플
y = ys[0]
print(y.data)

5


# STEP12 가변 길이 인수(개선 편)